In [1]:
import tensorflow as tf
import numpy as np
from components.positional import add_timing_signal_nd

In [5]:
embedding_dim = 80

In [23]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', input_shape=(50, 120, 3)))
model.add(tf.keras.layers.MaxPool2D(padding='same'))
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(padding='same'))
model.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 1), strides=(2, 1), padding='same'))
model.add(tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(1, 2), strides=(1, 2), padding='same'))
model.add(tf.keras.layers.Conv2D(512, 3, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Lambda(add_timing_signal_nd))
model.add(tf.keras.layers.Dense(embedding_dim))

AttributeError: module 'tensorflow' has no attribute 'to_float'

In [17]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])



In [18]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.